In [24]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [45]:
# amount of each categories for company bundle
Company_bundle = np.array([
    [5,3,1], # company A
    [7,3,2], # comapny B
    [2,1,1], # Company c
    [3,1,4] # company D
])

Company, bundle_profit_first, bundle_profit_second, bundle_profit_third, per_lb = gp.multidict({
    'A' : [2, 2.2, 2.1, 9],
    'B' : [3, 3.4, 3.2, 10],
    'C' : [1, 1.1, 1.05, 4],
    'D' : [2, 2.2, 2.1, 9]
})

# amount of supply, already subtracted from the company relationship require
# money get from small recycler 
Categories, amount, profit_single_first, profist_single_second, profist_single_third = gp.multidict({
    'paper' : [49000, 0.05, 0.06, 0.055],
    'glass' : [18000, 0.03, 0.03, 0.03],
    'plastic' : [36000, 0.02, 0.02, 0.02]
})


1) 

Total collection

15,000 pounds of paper

7,000 pounds of glass

14,000 pounds of plastic

In [56]:
company_weight = np.array(list(per_lb.values()))
company_profit_first = np.array(list(bundle_profit_first.values()))
company_profit_second = np.array(list(bundle_profit_second.values()))
company_profit_third = np.array(list(bundle_profit_third.values()))

single_profit_first = np.array(list(profit_single_first.values()))
single_profit_second = np.array(list(profist_single_second.values()))
single_profit_third = np.array(list(profist_single_third.values()))


available_materials = np.array(list(amount.values()))


# Create model
model = gp.Model(name='revenue_two_months_integer')
model.setParam('OutputFlag', 1)

# SIMPLIFIED APPROACH: Track bundles (can be fractional multiplier), materials are calculated

# Decision variables - bundles sold to each company (continuous to allow fractional)
bundles_first = model.addMVar(len(Company), vtype=GRB.CONTINUOUS, lb=100, name='bundles_first')
bundles_second = model.addMVar(len(Company), vtype=GRB.CONTINUOUS, lb=100, name='bundles_second')
bundles_third = model.addMVar(len(Company), vtype=GRB.CONTINUOUS, lb=100, name='bundles_third')

# Small recycler materials (integer pounds)
small_first = model.addMVar(len(Categories), vtype=GRB.INTEGER, lb=0, name='small_first')
small_second = model.addMVar(len(Categories), vtype=GRB.INTEGER, lb=0, name='small_second')
small_third = model.addMVar(len(Categories), vtype=GRB.INTEGER, lb=0, name='small_third')

# CONSTRAINTS: Material balance
# For each material: (bundles to companies) + (to small recyclers) = available

for j in range(len(Categories)):
    # Material j used by all companies in month 1
    materials_m1 = gp.quicksum(Company_bundle[i, j] * bundles_first[i] for i in range(len(Company)))
    
    # Material j used by all companies in month 2
    materials_m2 = gp.quicksum(Company_bundle[i, j] * bundles_second[i] for i in range(len(Company)))
    
    materials_m3 = gp.quicksum(Company_bundle[i,j] * bundles_third[i] for i in range(len(Company)) )
    # Material balance
    model.addConstr(
        materials_m1 + materials_m2 + materials_m3 + small_first[j] + small_second[j] + small_third[j] == available_materials[j],
        name=f"balance_{j}"
    )

# Revenue from large recyclers - MONTH 1
# For each company: (bundles * weight_per_bundle) * price_per_lb = bundles * price_per_bundle
first_large_revenue = gp.quicksum(
    company_profit_first[i] * bundles_first[i]
    for i in range(len(Company))
)

# Revenue from large recyclers - MONTH 2
second_large_revenue = gp.quicksum(
    company_profit_second[i] * bundles_second[i]
    for i in range(len(Company))
)
third_large_revenue =  gp.quicksum(
    company_profit_third[i] * bundles_third[i]
    for i in range(len(Company))
)


# Revenue from small recyclers
small_revenue_first = gp.quicksum(
    single_profit_first[j] * small_first[j]
    for j in range(len(Categories))
)

small_revenue_second = gp.quicksum(
    single_profit_second[j] * small_second[j]
    for j in range(len(Categories))
)


small_revenue_second = gp.quicksum(
    single_profit_third[j] * small_third[j]
    for j in range(len(Categories))
)


# Total revenue
total_revenue = first_large_revenue + second_large_revenue + third_large_revenue + small_revenue_first + small_revenue_second + small_revenue_second

model.setObjective(total_revenue, GRB.MAXIMIZE)

# Solve
print("\n" + "="*70)
print("SOLVING MODEL...")
print("="*70)
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("\n" + "="*70)
    print("OPTIMAL SOLUTION FOUND!")
    print("="*70)
    
    bundles_first_vals = bundles_first.X
    bundles_second_vals = bundles_second.X
    bundles_third_vals = bundles_third.X

    small_first_vals = small_first.X
    small_second_vals = small_second.X
    small_third_vals = small_third.X
    # MONTH 1
    print("\n" + "="*70)
    print("MONTH 1 RESULTS")
    print("="*70)
    
    print("\nLarge Recyclers (Month 1):")
    print("-" * 70)
    month1_large_revenue = 0
    
    for i, comp in enumerate(Company):
        bundles = bundles_first_vals[i]
        total_pounds = company_weight[i] * bundles
        revenue = company_profit_first[i] * bundles
        month1_large_revenue += revenue
        
        print(f"\nCompany {comp}:")
        print(f"  Bundles sold:       {bundles:.4f}")
        print(f"  Total weight:       {total_pounds:.2f} lbs")
        print(f"  Price/bundle:       ${company_profit_first[i]:.2f}")
        print(f"  Revenue:            ${revenue:.2f}")
        print(f"  Materials breakdown:")
        for j, cat in enumerate(Categories):
            material_lbs = Company_bundle[i, j] * bundles
            print(f"    {cat.capitalize():8}: {material_lbs:10.2f} lbs")
    
    print("\n" + "-" * 70)
    print("Small Recyclers (Month 1):")
    month1_small_revenue = 0
    for j, cat in enumerate(Categories):
        revenue = single_profit_first[j] * small_first_vals[j]
        month1_small_revenue += revenue
        print(f"  {cat.capitalize():8}: {small_first_vals[j]:10.0f} lbs @ ${single_profit_first[j]:.2f}/lb → ${revenue:.2f}")
    
    month1_total = month1_large_revenue + month1_small_revenue
    print(f"\nMonth 1 Total Revenue: ${month1_total:.2f}")
    
    # MONTH 2
    print("\n" + "="*70)
    print("MONTH 2 RESULTS")
    print("="*70)
    
    print("\nLarge Recyclers (Month 2):")
    print("-" * 70)
    month2_large_revenue = 0
    
    for i, comp in enumerate(Company):
        bundles = bundles_second_vals[i]
        total_pounds = company_weight[i] * bundles
        revenue = company_profit_second[i] * bundles
        month2_large_revenue += revenue
        
        print(f"\nCompany {comp}:")
        print(f"  Bundles sold:       {bundles:.4f}")
        print(f"  Total weight:       {total_pounds:.2f} lbs")
        print(f"  Price/bundle:       ${company_profit_second[i]:.2f}")
        print(f"  Revenue:            ${revenue:.2f}")
        print(f"  Materials breakdown:")
        for j, cat in enumerate(Categories):
            material_lbs = Company_bundle[i, j] * bundles
            print(f"    {cat.capitalize():8}: {material_lbs:10.2f} lbs")
    
    print("\n" + "-" * 70)
    print("Small Recyclers (Month 2):")
    month2_small_revenue = 0
    for j, cat in enumerate(Categories):
        revenue = single_profit_second[j] * small_second_vals[j]
        month2_small_revenue += revenue
        print(f"  {cat.capitalize():8}: {small_second_vals[j]:10.0f} lbs @ ${single_profit_second[j]:.2f}/lb → ${revenue:.2f}")
    
    month2_total = month2_large_revenue + month2_small_revenue
    print(f"\nMonth 2 Total Revenue: ${month2_total:.2f}")

    # MONTH 3
    print("\n" + "="*70)
    print("MONTH 3 RESULTS")
    print("="*70)
    
    print("\nLarge Recyclers (Month 3):")
    print("-" * 70)
    month3_large_revenue = 0
    
    for i, comp in enumerate(Company):
        bundles = bundles_third_vals[i]
        total_pounds = company_weight[i] * bundles
        revenue = company_profit_third[i] * bundles
        month3_large_revenue += revenue
        
        print(f"\nCompany {comp}:")
        print(f"  Bundles sold:       {bundles:.4f}")
        print(f"  Total weight:       {total_pounds:.2f} lbs")
        print(f"  Price/bundle:       ${company_profit_third[i]:.2f}")
        print(f"  Revenue:            ${revenue:.2f}")
        print(f"  Materials breakdown:")
        for j, cat in enumerate(Categories):
            material_lbs = Company_bundle[i, j] * bundles
            print(f"    {cat.capitalize():8}: {material_lbs:10.2f} lbs")
    
    print("\n" + "-" * 70)
    print("Small Recyclers (Month 3):")
    month3_small_revenue = 0
    for j, cat in enumerate(Categories):
        revenue = single_profit_third[j] * small_third_vals[j]
        month3_small_revenue += revenue
        print(f"  {cat.capitalize():8}: {small_third_vals[j]:10.0f} lbs @ ${single_profit_third[j]:.2f}/lb → ${revenue:.2f}")
    
    month3_total = month3_large_revenue + month3_small_revenue
    print(f"\nMonth 3 Total Revenue: ${month3_total:.2f}")
    
    # TOTAL RESULTS
    print("\n" + "="*70)
    print("TOTAL REVENUE (BOTH MONTHS)")
    print("="*70)
    print(f"\nMonth 1: ${month1_total:.2f}")
    print(f"Month 2: ${month2_total:.2f}")
    print(f"Month 3: ${month3_total:.2f}")
    print(f"TOTAL:   ${model.objVal:.2f}")
    
    # Material verification
    print("\n" + "="*70)
    print("MATERIAL USAGE VERIFICATION")
    print("="*70)
    print(f"{'Material':10} | {'Month 1':15} | {'Month 2':15} | {'Month 3':15} | {'Total Used':15} | {'Available':15} | {'Leftover':15}")
    print("-" * 95)
    
    for j, cat in enumerate(Categories):
        m1_to_companies = sum(Company_bundle[i, j] * bundles_first_vals[i] for i in range(len(Company)))
        m2_to_companies = sum(Company_bundle[i, j] * bundles_second_vals[i] for i in range(len(Company)))
        m3_to_companies = sum(Company_bundle[i, j] * bundles_third_vals[i] for i in range(len(Company)))
        m1_use = m1_to_companies + small_first_vals[j]
        m2_use = m2_to_companies + small_second_vals[j]
        m3_use = m3_to_companies + small_third_vals[j]
        total_use = m1_use + m2_use + m3_use
        leftover = available_materials[j] - total_use
        
        print(f"{cat.capitalize():10} | {m1_use:15.2f} | {m2_use:15.2f} | {m3_use:15.2f} | {total_use:15.2f} | {available_materials[j]:15.0f} | {leftover:15.2f}")

elif model.status == GRB.INFEASIBLE:
    print("\nModel is INFEASIBLE")
    print("Computing IIS (Irreducible Inconsistent Subsystem)...")
    model.computeIIS()
    print("\nInfeasible constraints:")
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"  {c.ConstrName}: {c}")
    print("\nInfeasible bounds:")
    for v in model.getVars():
        if v.IISLBound:
            print(f"  {v.VarName} >= {v.LB}")
        if v.IISUBound:
            print(f"  {v.VarName} <= {v.UB}")

elif model.status == GRB.UNBOUNDED:
    print("\nModel is UNBOUNDED - Check objective and constraints")
else:
    print(f"Status: {model.status}")



Set parameter OutputFlag to value 1

SOLVING MODEL...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3 rows, 21 columns and 45 nonzeros
Model fingerprint: 0xab26f36c
Variable types: 12 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e-02, 3e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [2e+04, 5e+04]
Presolve removed 0 rows and 14 columns
Presolve time: 0.00s
Presolved: 3 rows, 7 columns, 15 nonzeros
Variable types: 4 continuous, 3 integer (0 binary)
Found heuristic solution: objective 9644.0000000

Root relaxation: objective 2.795360e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N